In [2]:
# Genetic Algorithm With parameters
import numpy as np

# Parameters
population_size = 1000   # Number of individuals in each generation
num_variables = 5        # Number of parameters to optimize
num_parents = 500        # Number of parents selected to breed
num_generations = 1000   # Number of generations
crossover_rate = 0.8     # Probability of crossover
mutation_rate = 0.1      # Probability of mutation
mutation_range = [1, 5E5, 0.01, 1E-7, 5E-7, 5E-7]       # Range of mutation 
lower_bounds = [10, 50E6, 0.1, 0.5E-6, 10E-6, 50E-6]    # Bounds on parameters
upper_bounds = [1001, 500E6, 0.21, 500E-6, 500E-6, 500E-6]

# Test Solution
solution = np.array([55, 100E6, 0.5, 200E-6, 50E-6, 90E-6])

# Initialize population
no_pulses = np.random.randint(10, 1001, (population_size, 1))
f_0 = np.random.randint(50E6, 500E6, (population_size, 1))
amplitude = np.random.uniform(0.1, 0.21, (population_size, 1))
freq_sweep = np.random.uniform(0.5E-6, 500E-6, (population_size, 1))
duration = np.random.uniform(10E-6, 500E-6, (population_size, 1))
delay = np.random.uniform(50E-6, 500E-6, (population_size, 1))

population = np.hstack((no_pulses, f_0, amplitude, freq_sweep, duration, delay))

#This will be how the fitness is calculated After the code runs once, the csv files will be called and the efficency will be calculated 
#fitness = (0_echo[:, 2]-0_input_no_rephasing[:, 2])/0_input_cross[:,2]
#Note the fitness function does not work with the parameters
def calculate_fitness(population, solution):
    difference = 100 - np.abs(solution - population)
    fitness = np.sum(difference, axis=1) / (100 * population.shape[1])
    return fitness

def crossover(parents, crossover_rate, population_size):
    num_parents = parents.shape[0]
    num_variables = parents.shape[1]

    # Initialize offspring population array
    offspring_population = np.zeros((population_size, num_variables))
    current_count = 0

    while current_count < population_size:
        # Randomly shuffle the order of parents
        shuffled_indices = np.random.permutation(num_parents)
        shuffled_parents = parents[shuffled_indices, :]

        # Iterate over pairs of parents
        for i in range(0, num_parents, 2):
            if current_count >= population_size:
                break

            if i == num_parents - 1:
                offspring_population[current_count, :] = shuffled_parents[i, :]  # if odd number parents
                current_count += 1
            else:
                parent1 = shuffled_parents[i, :]
                parent2 = shuffled_parents[i + 1, :]

                if np.random.rand() < crossover_rate:
                    # Perform crossover
                    crossover_point = np.random.randint(1, num_variables)
                    offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                    offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
                else:
                    # No crossover, offspring are copies of parents
                    offspring1 = parent1
                    offspring2 = parent2

                # Add offspring to the population
                offspring_population[current_count, :] = offspring1
                current_count += 1

                if current_count < population_size:
                    offspring_population[current_count, :] = offspring2
                    current_count += 1

    return offspring_population

def mutate(new_population, mutation_rate, mutation_range):
    population_size = new_population.shape[0]
    num_variables = new_population.shape[1]

    mutated_population = new_population.copy()

    for i in range(population_size):
        for j in range(num_variables):
            if np.random.rand() < mutation_rate:
                mutation_value = (np.random.rand() - 0.5) * 2 * mutation_range[j]
                mutated_population[i, j] = mutated_population[i, j] + mutation_value
                mutated_population[i, j] = max(lower_bounds[j], min(upper_bounds[j], mutated_population[i, j]))

    return mutated_population

# Genetic Algorithm
for i in range(num_generations):
    # Fitness
    fitness = calculate_fitness(population, solution)

    # Selection
    parents = population[np.random.choice(population_size, num_parents, p=fitness/fitness.sum())]

    # Breeding Crossover and Mutation
    population = crossover(parents, crossover_rate, population_size)
    population = mutate(population, mutation_rate, mutation_range)

# Evaluation
fitness = calculate_fitness(population, solution)
max_value = np.max(fitness)
index_of_max = np.argmax(fitness)
min_value = np.min(fitness)
index_of_min = np.argmin(fitness)
avg_fit = np.mean(fitness)
best_fit = max_value
best = population[index_of_max, :]
worse_fit = min_value
worse = population[index_of_min, :]

print("Average Fitness:", avg_fit)
print("Best Fitness:", best_fit)
print("solution", solution)
print("Best Solution:", best)
print("Worst Fitness:", worse_fit)
print("Worst Solution:", worse)


Average Fitness: -665105.4645722047
Best Fitness: -661635.989619463
solution [5.5e+01 1.0e+08 5.0e-01 2.0e-04 5.0e-05 9.0e-05]
Best Solution: [5.94726903e+02 4.96981654e+08 1.05951179e-01 1.83137272e-04
 4.88384357e-04 2.50984423e-04]
Worst Fitness: -666667.2292309207
Worst Solution: [9.92148478e+02 5.00000000e+08 1.10544829e-01 1.83586560e-04
 4.92003914e-04 2.50523169e-04]
